The goal of this notebook is to generate sentences using vanilla recurrent neural networks

In [10]:
from datetime import datetime
import itertools
import numpy as np
import nltk
import os
import operator
import sys

In [3]:
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/Ajay/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /Users/Ajay/nltk_data...
[nltk_data]    |  

True

In [302]:
vocabulary_size = 8000 # Cuz it'll take forever otherwise
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

corpora_dir = "/Users/Ajay/nltk_data/corpora/state_union"

In [304]:
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print ("Reading Data...")

# Read all file paths in corpora directory
file_list = []
for root, _ , files in os.walk(corpora_dir):  
    for filename in files:
        file_list.append(os.path.join(root, filename))

sentences = []

for files in file_list:
    with open(files, 'r') as fin:
        try:
            str_form = fin.read().replace('\n', '')
            sentences.extend(nltk.sent_tokenize(str_form))
        except UnicodeDecodeError: 
            # Some sentences have wierd characters. Ignore them for now
            pass

# Get all sentences in all files
sentences[:5]

Reading Data...


['PRESIDENT GEORGE H.W.',
 "BUSH'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS ON THE STATE OF THE UNION  January 31, 1990Mr.",
 'President, Mr. Speaker, Members of the United States Congress: I return as a former President of the Senate and a former Member of this great House.',
 'And now, as President, it is my privilege to report to you on the state of the Union.',
 'Tonight I come not to speak about the state of the Government, not to detail every new initiative we plan for the coming year nor to describe every line in the budget.']

In [305]:
# Add sentence delimiters
sentences = [sentence_start_token + " " + x + " " + sentence_end_token for x in sentences ]

In [306]:
sentences[:5]

['SENTENCE_START PRESIDENT GEORGE H.W. SENTENCE_END',
 "SENTENCE_START BUSH'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS ON THE STATE OF THE UNION  January 31, 1990Mr. SENTENCE_END",
 'SENTENCE_START President, Mr. Speaker, Members of the United States Congress: I return as a former President of the Senate and a former Member of this great House. SENTENCE_END',
 'SENTENCE_START And now, as President, it is my privilege to report to you on the state of the Union. SENTENCE_END',
 'SENTENCE_START Tonight I come not to speak about the state of the Government, not to detail every new initiative we plan for the coming year nor to describe every line in the budget. SENTENCE_END']

In [307]:
#Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

print("Found " , len(word_freq.items()) ," unique words tokens.")

Found  18342  unique words tokens.


In [309]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1) # Get the most frequent words to construct vocab
index_to_word = [x[0] for x in vocab] # Extract word
index_to_word.append(unknown_token) # Add an extra token called "unknown" for the words in corpora, but not in vocab
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)]) # Create word-index map

print ("Using vocabulary size ", vocabulary_size)
print ("The least frequent word in our vocabulary is '",vocab[-1][0],
       "' and appeared ",vocab[-1][1]," times.")

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

Using vocabulary size  8000
The least frequent word in our vocabulary is ' three-year ' and appeared  2  times.


In [44]:
## Create the training data
# Every X represents a word. Every y represents a word that follows it in the sequence
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [310]:
# Print an training data example
x_example, y_example = X_train[10], y_train[10]
# print ( list(zip(x_example, y_example))) 
print("The 10th sentence input and expected output to every neuron looks like this\n")
print ( list(zip([index_to_word[x] for x in x_example], [index_to_word[y] for y in y_example])))

The 10th sentence input and expected output to every neuron looks like this

[('SENTENCE_START', 'new'), ('new', 'most'), ('most', 'of'), ('of', ','), (',', 'world'), ('world', 'Applause'), ('Applause', 'open'), ('open', 'the'), ('the', ','), (',', 'forward'), ('forward', 'of'), ('of', 'necessary'), ('necessary', 'the'), ('the', 'now'), ('now', '('), ('(', 'our'), ('our', 'democratic'), ('democratic', 'debt'), ('debt', 'the'), ('the', 'system'), ('system', 'economic'), ('economic', 'old'), ('old', 'we'), ('we', 'this'), ('this', 'aid'), ('aid', ','), (',', 'almost'), ('almost', 'of'), ('of', 'our'), ('our', 'you'), ('you', 'build'), ('build', 'in'), ('in', ','), (',', 'by'), ('by', 'that'), ('that', 'remain'), ('remain', '.'), ('.', 'SENTENCE_END')]


In [311]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [312]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim=50, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
        
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def calculate_total_loss(self, x, y):
        L = 0
        # For each sentence...
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            # We only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # Add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    def calculate_loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x,y)/N
    
    def sgd_step(self, x, y, learning_rate):
        # Calculate the gradients
        dLdU, dLdV, dLdW = self.bptt(x, y)
        # Change parameters according to gradients and learning rate
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
    
    def bptt(self, x, y):
        T = len(y)
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])              
                dLdU[:,x[bptt_step]] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        # Calculate the gradients using backpropagation. We want to checker if these are correct.
        bptt_gradients = model.bptt(x, y)
        # List of all parameters we want to check.
        model_parameters = ['U', 'V', 'W']
        # Gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # Get the actual parameter value from the mode, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print ("Performing gradient check for parameter ",pname," with size ",  np.prod(parameter.shape))
            # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # Save the original value so we can reset it later
                original_value = parameter[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                parameter[ix] = original_value + h
                gradplus = model.calculate_total_loss([x],[y])
                parameter[ix] = original_value - h
                gradminus = model.calculate_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # Reset parameter to original value
                parameter[ix] = original_value
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate The relative error: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # If the error is to large fail the gradient check
                if relative_error > error_threshold:
                    print ("Gradient Check ERROR: parameter= ",pname," ix=",ix)
                    print ("+h Loss: ", gradplus)
                    print ("-h Loss: ", gradminus)
                    print ("Estimated_gradient: ", estimated_gradient)
                    print ("Backpropagation gradient: ", backprop_gradient)
                    print ("Relative Error: ", relative_error)
                    return 
                it.iternext()
            print ("Gradient check for parameter ",pname," passed.")
            

In [313]:
np.random.seed(10)
model = RNN(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print (o.shape)
print (o)

(38, 8000)
[[0.00012467 0.00012468 0.00012489 ... 0.00012492 0.00012476 0.00012494]
 [0.00012558 0.00012582 0.00012467 ... 0.00012497 0.00012474 0.00012535]
 [0.00012561 0.00012434 0.00012451 ... 0.0001249  0.000125   0.00012543]
 ...
 [0.00012423 0.00012496 0.0001261  ... 0.00012443 0.00012564 0.00012578]
 [0.00012596 0.00012544 0.00012486 ... 0.00012441 0.0001262  0.00012466]
 [0.00012492 0.00012634 0.00012597 ... 0.00012599 0.00012423 0.00012585]]


The output at each timestep is a vector of vocabulary size, that denotes word probabilities. By taking the highest one, we get the next predicted word in the sequence. 

In [314]:
# Proving the predict function works
predictions = model.predict(X_train[10])
print (predictions.shape)
# Gibberish output. We're predicting the next words before even training. LOL
print (predictions)

(38,)
[7292 2356 6879 4319 1706 4335 1681 2973  141 6212 5687 5764 4708  141
 7814  497 1903 4702 5375 5720 1143  186 4372 3866 4107 1613 2859 1649
 5764  820 5791 1511 2286 2137 3265 6224 6477 6806]


In [315]:
print( [index_to_word[w] for w in predictions])

['favorably', 'reconstruction', 'characterize', 'armistice', 'elements', 'proportion', 'below', 'phase', 'support', 'torture', 'wholesale', 'wheels', 'announcing', 'support', 'enriching', 'commitment', 'changed', 'Steven', 'accurately', 'emphasized', 'quickly', 'come', 'advise', 'considerable', 'Reverend', 'saved', 'Whatever', 'provisions', 'wheels', 'hopes', 'proceeds', 'size', 'hungry', 'District', 'continuation', 'searching', 'characterized', 'wound']


In [316]:
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: ", np.log(vocabulary_size))
print ("Actual loss: ", model.calculate_loss(X_train[:1000], y_train[:1000]))

Expected Loss for random predictions:  8.987196820661973
Actual loss:  8.98714522074652


In [317]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter  U  with size  1000


/Users/Ajay/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:111: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter  U  passed.
Performing gradient check for parameter  V  with size  1000
Gradient check for parameter  V  passed.
Performing gradient check for parameter  W  with size  100
Gradient check for parameter  W  passed.


In [318]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print (time,": Loss after num_examples_seen=",num_examples_seen," epoch=",epoch,": ",loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to ", learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [319]:
np.random.seed(10)
model = RNN(vocabulary_size)
train_with_sgd(model, X_train, y_train)

2018-11-21 13:29:43 : Loss after num_examples_seen= 0  epoch= 0 :  8.98714522074652
2018-11-21 13:30:58 : Loss after num_examples_seen= 1000  epoch= 5 :  6.098045359540115
2018-11-21 13:32:11 : Loss after num_examples_seen= 2000  epoch= 10 :  5.724233313979789
2018-11-21 13:33:19 : Loss after num_examples_seen= 3000  epoch= 15 :  5.553371330132457
2018-11-21 13:34:31 : Loss after num_examples_seen= 4000  epoch= 20 :  5.41661623732687
2018-11-21 13:35:40 : Loss after num_examples_seen= 5000  epoch= 25 :  5.318534725306403
2018-11-21 13:36:50 : Loss after num_examples_seen= 6000  epoch= 30 :  5.236015662924755
2018-11-21 13:38:00 : Loss after num_examples_seen= 7000  epoch= 35 :  5.171466140436469
2018-11-21 13:39:15 : Loss after num_examples_seen= 8000  epoch= 40 :  5.106002083005647
2018-11-21 13:40:29 : Loss after num_examples_seen= 9000  epoch= 45 :  5.0614697968705435
2018-11-21 13:41:42 : Loss after num_examples_seen= 10000  epoch= 50 :  5.0192272785816545
2018-11-21 13:43:00 : Los

In [321]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while (not new_sentence[-1] == word_to_index[sentence_end_token]):
        next_word_probs, states = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
        
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 10
senten_min_length = 7

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print (" ".join(sent))

I n't other billion when , you world of , by most `` 's a an of , supply .
I with economic taking his peace-loving the year child the enduring was ( switch , value for enemies , American of , also .
burden , everything benefiting of , cycle we Our , Congress of more was , ago to , But monopolies to today teachers work , society This in he to pursue to well to without , well of retirement and : fall but our major . national , time every children child we but more this these expenditures our Capitol am in , million fiscal developed employment , live the their between we these hand means number progress in are of United exploring we agricultural will net to national civilian .
House specific Applause , system had the freedom the in there economy standstill our Congress will who .
new leave at to independent in a until the and -- in create why very living problems bring serve make now , believe of no to 950 must a changes the important you world me , these be or be set to develop to costs 

In [334]:
def save_model(outfile, model):
    U, V, W = model.U, model.V, model.W
    np.savez(outfile, U=U, V=V, W=W)
    print ("Saved model parameters to: ", outfile)

def load_model(path, model):
    npzfile = np.load(path)
    U, V, W = npzfile["U"], npzfile["V"], npzfile["W"]
    model.hidden_dim = U.shape[0]
    model.word_dim = U.shape[1]
    model.U = U
    model.V = V
    model.W = W
    print ("Loaded model parameters from ",path,". hidden_dim=",U.shape[0]," word_dim=", U.shape[1])

In [326]:
save_model("learned_model", model)

Saved model parameters to:  learned_model


In [331]:
mm = RNN(vocabulary_size)
load_model("learned_model.npz", mm)

Loaded model parameters from  learned_model.npz . hidden_dim= 50  word_dim= 8000


In [333]:
train_with_sgd(mm, X_train, y_train) # Continue training where we left off

2018-11-21 14:13:41 : Loss after num_examples_seen= 0  epoch= 0 :  4.484920437574073
2018-11-21 14:15:12 : Loss after num_examples_seen= 1000  epoch= 5 :  4.538133982986044
Setting learning rate to  0.0025
2018-11-21 14:16:34 : Loss after num_examples_seen= 2000  epoch= 10 :  4.467856978524159
2018-11-21 14:17:55 : Loss after num_examples_seen= 3000  epoch= 15 :  4.439540265729607
2018-11-21 14:19:19 : Loss after num_examples_seen= 4000  epoch= 20 :  4.377615473275601
2018-11-21 14:20:43 : Loss after num_examples_seen= 5000  epoch= 25 :  4.341018487704967
2018-11-21 14:22:04 : Loss after num_examples_seen= 6000  epoch= 30 :  4.342171445984887
Setting learning rate to  0.00125
2018-11-21 14:23:28 : Loss after num_examples_seen= 7000  epoch= 35 :  4.192869267977804
2018-11-21 14:24:49 : Loss after num_examples_seen= 8000  epoch= 40 :  4.196551819799632
Setting learning rate to  0.000625
2018-11-21 14:26:13 : Loss after num_examples_seen= 9000  epoch= 45 :  4.090430858002756
2018-11-21 14